# Markov Decision Processes
Covers formalism, etc... These differ from bandits in two key ways: 

* There is modeled dependence on "context" - for bandits, we want to estimate the value $q_*(a)$, but in an MDP we also want to estimate the value as a function of state or context: $q_*(s, a)$.

* There is dependence on sequential decisions because our actions can change subsequent state...

Generally, an agent (learner or decision maker) interacts with an environment.  At each time step $t$ (can be continuous), the agent selects an action $a \in \mathcal{A}(s)$ (note dependence of action set on state).  The environment evolves in response to the action (i.e., subsequent state can depend on the action taken) => $S_{t+1}$, and also provides some scalar reward $R_{t+1} \in \mathbb{R}$ in response to the starting action and state, $A_t$ and $S_t \in \mathcal{S}$.  

An interesting practical question is - where do we draw the boundary between agent and environment?  "The general rule we follow is that anything that cannot be changed arbitrarily by the agent is considered to be outside of it and thus part of the environment.  We do not assumed that everything in the environment is unknown to the agent."  This is an important principle in formulating MDPs - the boundary is one of control, not knowledge!  For instance, in a robot control system, we probably want to draw the boundary at the signals sent to actuators since that is all the agent can arbitrarily control...  This is so even if the robot has pretty precise and accurate sensors telling us how many steps were taken by a step motor, for instance.  "the agent-environment boundary represents the limit of the agent's absolute control, not of its knowledge". 

This chapter introduces lots of notation so there's not much to be done for it except to slog through...


## The reward hypothesis
They make a big deal of this, and Michael Littman of Brown has a thoughtful video about this that is worth checking out...  

>That all of what we mean by goals and purposes can be well thought of as the maximization of the expected value of the cumulative sum of a received scalar signal (called reward).

This is a big assumption and a _sine qua none_ of RL, and we can often try to make this approximately true by expanding our notion of state...  Note that the reward signal is not where we want to impart prior knowledge to the agent about _how_ goals should be achieved!  

## MDP Dynamics 
Dynamics in MDPs refers to how the environment evolves over time in response to actions taken by the agent.  This evolution is stochastic, and is assumed to be governed _completely_ by a probability distribution: 

$$
p(s', r | s, a) = Pr{S_t = s', R_t = r | S_{t-1} = s, A_{t-1} = a} \text{  } \forall s', s \in \mathcal{S}, a \in \mathcal{A}(s)
$$

i.e., the dynamics specify a distribution over rewards and states conditional on starting states and actions.  Thus, 
$$
\sum_{s' \in \mathcal{S}} \sum_{r \in \mathbb{R}} p(s', r | s, a) = 1 \text{    } \forall s \in \mathcal{S}, a \in \mathcal{A}(s)
$$

This dependence only on the current state is why we refer to this as a Markov process.  "This is best viewed as a restriction not on the decision process, but on the _state_.  The state must include information about all aspects of the past agent-environment interaction that make a difference for the future".  In practice, we can usually make this at least approximately true by simply expanding our definition of state...

## Things defined in terms of the dynamics

### State transition probabilities
We are probably used to thinking of Markov processes as governed by a state transition function.  Given the dynamics function, we can do the usual manipulations of probability distributions to arrive at various interesting things, like the implied state transition probabilities.  This is a probability distribution over subsequent states given a starting state and action, and we can get this by marginalizing the dynamics over rewards: 

$$
p(s' | s, a) = Pr{S_t = s' | S_{t-1} = s, A_{t-1} = a} = \sum_{r \in \mathbb{R}} p(s', a | s, a)
$$

### Expected rewards for state-action pairs
It would also be interesting to know what the expected reward is for taking action $a$ in state $s$...  If we have the dynamics function, we can calculate this: 

$$
r(s, a): \mathcal{S} \times \mathcal{A} \rightarrow \mathbb{R} = \
\mathbb{E}[R_t | S_{t-1} = s, A_{t-1} = a] = \sum_{r \in \mathbb{R}} \sum_{s' \in \mathcal{S}} r p(s', r|s, a)
$$

We can also ask more granular questions like, what is the expected reward if we start in $s$, take action $a$, and end up in state $s'$?  

$$
\begin{align}
r(s, a, s'): \mathcal{S} \times \mathcal{A} \times \mathcal{S} \rightarrow \mathbb{R} & = \
\mathbb{E}[R_t | S_{t-1} = s, A_{t-1} = a, S_t = s'] \\
& = \sum_{r \in \mathbb{R}}  r \frac{p(s', r|s, a)}{p(s'|s,a)}
\end{align}
$$

This expectation is defined in terms of the dynamics model and the state transition model. 

## Returns and episodes
### Rewards and episodic taks
How do we define the reward the agent is trying to maximize?  Say our task is something like, playing games of chess.  Each game has a natural end state, after which we probably want to reset the state to an initial state (or more generally, a draw from a distribution of initial states).  When the task is naturally broke up into subsequences, we call the subsequences _episodes_, and we formulate the problem to have a special _terminal state_ defining the ends of episodes.  These number of steps it takes to reach the terminal state is a random variable, $T$.  In this setting, it is also natural to define the _return_ at time $t$ as the sum of rewards received in times steps from $t$ up to and including $T$ (note - this elides a key question of what actions we take in those time steps!): 

$$
G_t = R_{t+1} + R_{t+2} + \dots + R_{T}
$$

### Continuous tasks
What do we do if our task is continuous, i.e., there isn't a good way to break it up into subsequences?  In other words, $T \rightarrow \infty$?  This immediately presents a problem - the return as we defined it is unbounded, and there is no good basis for saying that following one "policy" versus another is better or worse (we could introduce an arbitrary cut off point where we decide but that defeats ignores the possibility that we could do better with a bit more delayed gratification).  In this case, we use a _discount factor_, $0 \le \gamma < 1$ and define the discounted return: 

$$
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \dots = \sum_{k=0}^{\infty} \gamma^k R_{t + k + 1}
$$

As long as $R_t$ is bounded, this is finite, and has upper bound $\frac{\sup{\{R\}}}{1 - \gamma}$ since $\sum_{k=0}^{\infty} \gamma^k = \frac{1}{1-\gamma}$.  Practically, bigger $\gamma$ encourages longer term considerations; $\gamma = 0$ is very short sighted and we are treating this like a bandit problem.  On the other hand, $\gamma = 1$ gets us back to the definition of return we used for episodic tasks.

Note that we can write the return recursively: 

$$
G_t = R_{t+1} + \gamma G_{t+1}
$$

### Unified notation
In practice, we can use the notation for continuous tasks for episodic tasks by adopting a convention of a terminal absorbing state that transitions only to itself and has a reward of zero.  Then the math works out fine.  This saves us from introducing another index over episodes, which would be a pain in the ass.

## Policies and Value Functions
These are the key objects we'll work with...  The first object we define is a _policy_.  A policy, usually denoted $\pi$, is a stochastic function mapping states to a distribution over actions to take in that state: $\pi : \mathcal{S} \rightarrow \mathcal{A} = Pr(A = a | s) \forall s \in \mathcal{S}, a \in \mathcal{A}$.  Given a policy, we define two _value functions_, always with respect to some policy (often implicit).  

<b>Exercise 3.11.</b>  If the current state is $S_{t} = s$, and actions are selected according to $\pi$, then the expectation of the reward for this timestep is $R_{t+1}$ is: 

$$
\mathbb{E}_{\pi, p} [R_{t+1}] = \sum_{a \in \mathcal{A}} \sum_{s' \in \mathcal{S}} r \pi(a|s) p(r, s'|s, a)
$$

There are two _value functions_ we usually think about in RL.  The first, the <b>_value function</b> for policy_ $\pi, v_{\pi}$ maps states to expected return under some policy starting from state $s$:

$$
\begin{align}
v_{\pi}(s) & = \mathbb{E}_{\pi}[G_t | S_t = s] \\
& = \mathbb{E}_{\pi} \left[ \sum_{k=0}^{\infty} \gamma^k R_{t+k+1} | S_t = s \right] \text{   } \forall s \in \mathcal{S}
\end{align}
$$

We also work with a function called the <b>_action-value function</b> for policy_ $\pi$, mapping state-action pairs to expected returns if we take action $a$ from state $s$ and follow policy $\pi$ after that: 

$$
\begin{align}
q_{\pi}(s,a) & = \mathbb{E}_{\pi} \left[ G_t | S_t = s, A_t = a \right] \\
 & = \mathbb{E}_{\pi} \left[ \sum_{k=0}^{\infty} \gamma^k R_{t+k_1} | S_t=s, A_t=a \right]
\end{align}
$$


<b>Exercise 3.12</b> Give an equation for $v_{\pi}(s)$ in terms of $q_{\pi} \text{ and } \pi$.  
$$
v_{\pi}(s) = \sum_{a \in \mathcal{A}} \pi(a|s) q_{\pi}(s,a)
$$

<b>Exercise 3.13</b> Give an equation for $q_{\pi}$ in terms of $v_{\pi} \text{ and } p$.

$$
q_{\pi}(s, a) = \sum_{s' \in \mathcal{S}} p(s', r|s, a) \left(r + \gamma v_{\pi}(s') \right) 
$$

The value functions $v_{\pi}(s)$ and $q_{\pi}(s,a)$ can be estimated from experience given enough data, computation and memory.  In particular, if we have a finite MDP with discrete actions, states and rewards, we can store estimates as tables, and just count up things as they occur while we are following policy $\pi$.  This method is called _Monte Carlo estimation$, and works for tabular problems pretty well, but gets impractical for many problems of interest.  For the latter problems, we'll have to use parameterized function approximators of various sorts.  This may be the case even for finite MDPs because the state space and action space may simply be too big!

### Recursive definitions of the value function
An important property of the value function in MDPs is that we can define it recursively, similarly to our recursive definition of returns...  This recursive definition is called the Bellman equation for $v_{\pi}$:

$$
\begin{align}
v_{\pi} & = \mathbb{E}_{\pi} \left[ G_t | S_t = s \right] \\
 & = \mathbb{E}_{\pi} \left[ R_t + \gamma G_{t+1} | S_t = s \right] \\
 & = \sum_a \pi(a|s) \sum_{s'} \sum_r p(s',r|s,a) \left[ r + \gamma \mathbb{E}_{\pi} [G_{t+1}|S_{t+1}=s'] \right] \\
 & = \sum_a \pi(a|s) \sum_{s'} \sum_r p(s',r|s,a) \left[ r + \gamma v_{\pi}(s') \right]
\end{align}
$$

<b>Exercise 3.15 & 3.16</b> What happens to the value function if we add a constant $c$ to each reward?  In the continuous task setting, we simply shift the value function by $\frac{c}{1-\gamma}$ - this gets distributed out of any sums since none of the probabilities are affected by this, and we are following a fixed policy $\pi$.  In the episodic setting, we may have some impact because we are dealing with finite sums, and some policies may have shorter or longer episodes in expectation than other policies...

<b>Exercise 3.17</b> What is the Bellman equation for action values?

$$
\begin{align}
q_{\pi}(s,a) &= \sum_{r, s' \in \mathcal{S}} p(s', r|s, a) \left(r + \gamma v_{\pi}(s') \right) \\
& = \sum_{r, s' \in \mathcal{S}} p(s', r|s, a) \left(r + \gamma \sum_{a' \in \mathcal{A}} \pi(a'|s') q_{\pi}(s',a') \right) \\
\end{align}
$$

## Optimal policies and value functions
For finite MDPs we can define an optimal policy as follows.  Value functions define a partial ordering over policies - $\pi \ge \pi' \iff v_{pi}(s) \ge v_{pi'}(s) \text{   } \forall s \in \mathcal{S}$.  Note that this is for all states, but this isn't too big a deal - if we have policies disagreeing on states, we can make another policy that is at least as good as both on both states... So there is always at least one policy that is at least as good as all other policies.  This is an _optimal policy_. There can be more than one, but we'll refer to this by $\pi_*$.  All optimal policies have the same value function:

$$
v_*(s) = \max_{\pi} v_{\pi}(s) \text{   } \forall s \in \mathcal{S}
$$

Similarly, they all share the same _optimal action-value function_: 

$$
\begin{align}
q_*(s,a) & = \max_{\pi} q_{\pi}(s,a) \\
& = \mathbb{E} \left[ R_{t+1} + \gamma v_*(S_{t+1}) | S_t = s, A_t = a \right]
\end{align}
$$





### The Bellman optimality equations
We can derive recursive relations for the value and action-value functions.  These are also described as consistency requirements for obvious reasons.

For the value function, it's easiest to start with the idea that the value of a state under an optimal policy must be equal to the expected return of the best action available in that state.  Using exercise 3.17, 

$$
\begin{align}
v_*(s) & = \max_{a \in \mathcal{A}(s)} q_{\pi_*}(s,a) \\
& = \max_{a \in \mathcal{A}(s)} \sum_{r, s' \in \mathcal{S}} p(s', r|s, a) \left(r + \gamma v_{*}(s') \right) \\
\end{align}
$$

For the action-value function, 

$$
\begin{align}
q_*(s,a) & = \mathbb{E} \left[ R_{t+1} + \gamma \max_{a'} q_*(S_{t+1}, a') | S_t = s, A_t = a \right] \\
& = \sum_{r, s'} p(s', r|s,a) \left( r + \gamma \max_{a'} q_*(s', a') \right)
\end{align}
$$

<b>Exercises 3.25</b>  What is $v_*$ in terms of $q_*$?

$$
v_*(s) = \max_{a} q_*(s,a)
$$

<b>Exercise 3.26</b>  What is $q_*$ in terms of $v_*$ and $p$?  This is the expected value of taking $a$ from state $s$, plus the expected return of following $v_*$ from the resulting state:

$$
q_*(s,a) = \sum_{r, s'} p(s', r| s, a) \left( r + \gamma v_*(s') \right)
$$

<b>Exercise 3.27</b> What is $\pi_*$ in terms of $q_*$?  We just pick the $a$ that has the highest expected return from state $s$ under $q_*$:

$$
\pi_*(s) = \underset{a}{argmax} \text{ } q_*(s,a)
$$

<b>Exercise 3.28</b> What is $\pi_*(s)$ in terms of $v_*$ and $p$?  

$$
\pi_*(s) = \underset{a}{argmax} \text{ } p(s', r|s,a) \left( r + \gamma v_*(s') \right)
$$

### From optimal value functions to optimal policies
Once we have an optimal value function $v_*$, we can get an optimal policy.  For each state $s$, there are one or more actions at which the maximum is attained in the Bellman optimality equation.  We just pick from one of these actions and we will have an optimal policy (note that we can have more than one!).  This is analogous to a one-step search - we can do a one-step search into the future - we can do this _greedy_ search because we know that the longer term reards are already baked into value functions!  

If we have $q_*$, it's even easier - for any state $s$, we just pick from $a \in argmax_a q_*(s,a)$.  This is also like a greedy search that works out because value functions bake in expected future rewards.

However, note that solving the Bellman equations to get $v_*$ or $q_*$ is rarely directly useful in realistic problems - it relies on three assumptions that are rarely all true in practice: 

1. We have a good dynamics model (ha!)

2. We have computational resources to do the search 

3. the Markov property really holds.  

Even in a game like backgammon, where assumptions 1 and 3 hold, there are simply too many ($10^{20}$) states!  So in practice, we find approximate solutions.  In particular, RL methods that use experienced transitions in place of knowledge of the expected transitions are clearly approximations.  Luckily, for many problems, we can focus efforts on state-action pairs with lots of support for the state...  This is one difference between RL and traditional optimal control approaches to solving MDPs.  



## Conclusion
It's worth reading the historical notes about MDPs and RL - there are some fascinating connections going back to the origins of the theory of optimal control to classical mechanics that would be fun to chase down.  In particular, it turns out that the Bellman equations for the continuous time case are the Hamilton-Jacobi equations, and that the _action_ has a direct analogy to the action in Hamiltonian mechanics, which is wild but not crazy - consider that the action functional maps an entire trajectory to a scalar so you can view dynamics as making a greedy choice that minimizes the action functional!  Even turns out that the theoretical analysis of dynamic programming also uses ation-value functions.  